# CS483 - Colab 2
## Frequent Pattern Mining in Spark

In [1]:
from IPython.display import Image
print("Colab 2 Mascot")
Image(url='https://cdn.dribbble.com/users/222579/screenshots/1654898/stubby-ben-rex-roll.gif',width=150)

Colab 2 Mascot


### Setup

Let's set up Spark on your Colab environment.  Run the cell below!

In [2]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.3-py2.py3-none-any.whl size=317840625 sha256=c5993c6c41948e003ece393ccc881e662325fe71aa8919179061d5faebfd4a6f
  Stored in directory: /root/.cache/pip/wheels/1b/3a/92/28b93e2fbfdbb07509ca4d6f50c5e407f48dce4ddbda69a4ab
Successfully built pyspark
The following additional packages will be installed:
  libxtst6 openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra fonts-nanum fonts-ipafont-gothic
  fonts-ipafont-mincho fonts-wqy-microhei fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  libxtst6 openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 39.6 MB of archives.
After this operation, 144 MB of additional disk space will be used.
Selecting previously unselected package

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Now we authenticate a Google Drive client to download the file we will be processing in our Spark job.

**Make sure to follow the interactive instructions.**

In [5]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [6]:
id='1dhi1F78ssqR8gE6U-AgB80ZW7V_9snX4'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('products.csv')

id='1KZBNEaIyMTcsRV817us6uLZgm-Mii8oU'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('order_products__train.csv')

If you executed the cells above, you should be able to see the dataset we will need for this Colab under the "Files" tab on the left panel.

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

Let's initialize the Spark context.

In [8]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

### Your task

If you run successfully the setup stage, you are ready to work with the **3 Million Instacart Orders** dataset. In case you want to read more about it, check the [official Instacart blog post](https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2) about it, a concise [schema description](https://gist.github.com/jeremystan/c3b39d947d9b88b3ccff3147dbcf6c6b) of the dataset, and the [download page](https://www.instacart.com/datasets/grocery-shopping-2017).

In this Colab, we will be working only with a small training dataset (~131K orders) to perform fast Frequent Pattern Mining with the FP-Growth algorithm.

In [9]:
products = spark.read.csv('products.csv', header=True, inferSchema=True)
orders = spark.read.csv('order_products__train.csv', header=True, inferSchema=True)

In [10]:
products.printSchema()

root
 |-- product_id: integer (nullable = true)
 |-- product_name: string (nullable = true)
 |-- aisle_id: string (nullable = true)
 |-- department_id: string (nullable = true)



In [11]:
orders.printSchema()

root
 |-- order_id: integer (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- add_to_cart_order: integer (nullable = true)
 |-- reordered: integer (nullable = true)



Use the Spark Dataframe API to join 'products' and 'orders', so that you will be able to see the product names in each transaction (and not only their ids).  Then, group by the orders by 'order_id' to obtain one row per basket (i.e., set of products purchased together by one customer).

In [14]:
''' 2 lines of code expected '''
# YOUR CODE HERE
joined = orders.join(products, products.product_id == orders.product_id)
joined_grouped = joined.groupBy('order_id').agg(collect_set('product_name').alias('products'))
# joined_grouped.show()

In this Colab we will explore [MLlib](https://spark.apache.org/mllib/), Apache Spark's scalable machine learning library. Specifically, you can use its implementation of the [FP-Growth](https://spark.apache.org/docs/latest/ml-frequent-pattern-mining.html#fp-growth) algorithm to perform efficiently Frequent Pattern Mining in Spark.
Use the Python example in the documentation, and train a model with

```minSupport=0.01``` and ```minConfidence=0.5```



In [15]:
''' 3 lines of code expected '''
# YOUR CODE HERE
from pyspark.ml.fpm import FPGrowth
fpGrowth = FPGrowth(itemsCol="products", minSupport=0.01, minConfidence=0.5)
model = fpGrowth.fit(joined_grouped)

Compute how many frequent itemsets and association rules were generated by running FP-growth alongside visalizing top frequent itemsets and association rules.


In [21]:
''' 5 lines of code in total expected but can differ based on your style. for sub-parts of the question, creating different cells of code would be recommended.'''
# YOUR CODE HERE
print("Number of frequent itemsets: ", model.freqItemsets.count())
print("Number of association rules: ", model.associationRules.count())
print("\nTop frequent itemsets and Association Rules")
model.freqItemsets.sort(desc("freq")).show()
model.associationRules.sort(desc("confidence")).show()

Number of frequent itemsets:  120
Number of association rules:  0

Top frequent itemsets and Association Rules
+--------------------+-----+
|               items| freq|
+--------------------+-----+
|            [Banana]|18726|
|[Bag of Organic B...|15480|
|[Organic Strawber...|10894|
|[Organic Baby Spi...| 9784|
|       [Large Lemon]| 8135|
|   [Organic Avocado]| 7409|
|[Organic Hass Avo...| 7293|
|      [Strawberries]| 6494|
|             [Limes]| 6033|
|[Organic Raspberr...| 5546|
|[Organic Blueberr...| 4966|
|[Organic Whole Milk]| 4908|
|  [Organic Cucumber]| 4613|
|  [Organic Zucchini]| 4589|
|[Organic Yellow O...| 4290|
|    [Organic Garlic]| 4158|
|[Seedless Red Gra...| 4059|
|         [Asparagus]| 3868|
|[Organic Grape To...| 3823|
| [Organic Red Onion]| 3818|
+--------------------+-----+
only showing top 20 rows

+----------+----------+----------+----+-------+
|antecedent|consequent|confidence|lift|support|
+----------+----------+----------+----+-------+
+----------+----------+

Now retrain the FP-growth model changing only
```minsupport=0.001```
and compute how many frequent itemsets and association rules were generated.


In [22]:
''' 5 lines of code in total expected but can differ based on your style. for sub-parts of the question, creating different cells of code would be recommended.'''
# YOUR CODE HERE
fpGrowth = FPGrowth(itemsCol="products", minSupport=0.001, minConfidence=0.5)
model = fpGrowth.fit(joined_grouped)
print("Number of frequent itemsets: ", model.freqItemsets.count())
print("Number of association rules: ", model.associationRules.count())
print("\nTop frequent itemsets and Association Rules")
model.freqItemsets.sort(desc("freq")).show()
model.associationRules.sort(desc("confidence")).show()

Number of frequent itemsets:  4444
Number of association rules:  11

Top frequent itemsets and Association Rules
+--------------------+-----+
|               items| freq|
+--------------------+-----+
|            [Banana]|18726|
|[Bag of Organic B...|15480|
|[Organic Strawber...|10894|
|[Organic Baby Spi...| 9784|
|       [Large Lemon]| 8135|
|   [Organic Avocado]| 7409|
|[Organic Hass Avo...| 7293|
|      [Strawberries]| 6494|
|             [Limes]| 6033|
|[Organic Raspberr...| 5546|
|[Organic Blueberr...| 4966|
|[Organic Whole Milk]| 4908|
|  [Organic Cucumber]| 4613|
|  [Organic Zucchini]| 4589|
|[Organic Yellow O...| 4290|
|    [Organic Garlic]| 4158|
|[Seedless Red Gra...| 4059|
|         [Asparagus]| 3868|
|[Organic Grape To...| 3823|
| [Organic Red Onion]| 3818|
+--------------------+-----+
only showing top 20 rows

+--------------------+--------------------+------------------+------------------+--------------------+
|          antecedent|          consequent|        confidence|

To conclude, go to Gradescope and read the remaining questions. We will ask you to inspect the resulting dataframes, and report a few results.



In [ ]:
# YOUR CODE HERE

Once you obtained the desired results, **head over to Gradescope and submit your solution for this Colab**!